In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
    model="llama3-8b-8192"
)

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-MiniLM-L6-v2",
)

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
os.environ["PINECONE_API_KEY"] = os.environ["PINECONE_API_KEY"]
index_name = "storeagent"

vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embedding,
)

In [6]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

file_path = "info.txt"

loader = TextLoader(file_path=file_path, encoding="utf-8")
documents = loader.load()

In [8]:
len(documents)

1

In [9]:
documents

[Document(page_content='## Where is the store located?\n- The Store is located in Dharan sunsari district Nepal.\n\n## Who own the store?\n- The owner is Manoj Baniya.\n\n## What is the name of the store?\n- The name of the store is All Electronics store.\n\n## What do the store sell?\n- The store sell all kinds of electronics including smartphones, camera, laptops, pc, calculators...\n\n## What are the delivery options\n- The delivery  options is available all over Nepal and the delivery charge is Rs. 120 standard price and delivery is free inside Dharan area.\n\n## How long the delivery takes\n- The delivery takes about 2-3 days for all over Nepal and if you are within Dharan Delivery will be the same day you order product.\n\n## Is Return Policy available\n- User can return the products within 3 days after they received their product.\n\n## Return rule\n- The product should be in the same condition as when it was purchased before.\n\n## What are the payment options\n- Online payment

In [12]:
text_splitter = CharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=0
)
docs = text_splitter.split_documents(documents=documents)

In [13]:
len(docs)

10

In [14]:
docs

[Document(page_content='## Where is the store located?\n- The Store is located in Dharan sunsari district Nepal.\n\n## Who own the store?\n- The owner is Manoj Baniya.', metadata={'source': 'info.txt'}),
 Document(page_content='## What is the name of the store?\n- The name of the store is All Electronics store.', metadata={'source': 'info.txt'}),
 Document(page_content='## What do the store sell?\n- The store sell all kinds of electronics including smartphones, camera, laptops, pc, calculators...', metadata={'source': 'info.txt'}),
 Document(page_content='## What are the delivery options\n- The delivery  options is available all over Nepal and the delivery charge is Rs. 120 standard price and delivery is free inside Dharan area.', metadata={'source': 'info.txt'}),
 Document(page_content='## How long the delivery takes\n- The delivery takes about 2-3 days for all over Nepal and if you are within Dharan Delivery will be the same day you order product.', metadata={'source': 'info.txt'}),


In [16]:
vectorstore

In [17]:
vectorstore.add_documents(docs)

['1132f76a-c675-4157-b497-0bc3db3a8d12',
 'ec0ebdcb-f562-4548-938f-4a64cbfca0b0',
 '5d534793-f857-4fd8-8a2b-e91b9685ebbb',
 'c5fdf54d-fb6c-40d8-8330-31f527fdb9e2',
 'a6cff85b-0e1d-4462-aecd-2c7841bac782',
 '9f07a68f-83bb-4b4b-9238-da35d73181ac',
 'e195d4ca-10df-4374-a0dc-2a746448e659',
 '589899b4-e7d2-4d63-80e0-ff1b69aa8d08',
 'c0eb2ecb-3234-476d-9879-834977044f58',
 'e0373f39-3d32-44c7-abf3-baccdd72bdfb']

In [19]:
vectorstore.similarity_search("location of store", k=2)

[Document(page_content='## Where is the store located?\n- The Store is located in Dharan sunsari district Nepal.\n\n## Who own the store?\n- The owner is Manoj Baniya.', metadata={'source': 'info.txt'}),
 Document(page_content='## What is the name of the store?\n- The name of the store is All Electronics store.', metadata={'source': 'info.txt'})]

In [20]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k":2
    }
)
retriever.invoke("Delivery options")

[Document(page_content='## What are the delivery options\n- The delivery  options is available all over Nepal and the delivery charge is Rs. 120 standard price and delivery is free inside Dharan area.', metadata={'source': 'info.txt'}),
 Document(page_content='## How long the delivery takes\n- The delivery takes about 2-3 days for all over Nepal and if you are within Dharan Delivery will be the same day you order product.', metadata={'source': 'info.txt'})]

In [32]:
system_prompt = (
    "Given a chat history and the latest user question"
    "Which might reference context in the chat history,"

"formulate a standalone question which can be understood"
"without the chat history. Do not answer the question just"
"reformulate it if needed and otherwise return it as is"
"if asked any normal question answer it")

In [33]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [34]:
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    prompt
)

In [35]:
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001F0D0172CC0>, search_kwargs={'k': 2}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user questionWhich might reference context in the chat history,formulate a standalone question which can be understoodwithout the chat hist

In [36]:
history_aware_retriever.invoke({
    "chat_history": [
        "what is the location of the store",
        "Dharan"
    ],
    "input": "Delivery options"
})

[Document(page_content='## What are the delivery options\n- The delivery  options is available all over Nepal and the delivery charge is Rs. 120 standard price and delivery is free inside Dharan area.', metadata={'source': 'info.txt'}),
 Document(page_content='## What are the payment options\n- Online payment with khalti, esewa or Cash On Delivery is Available in our store.\n\n### SQL queries', metadata={'source': 'info.txt'})]

In [37]:
chat_system_prompt = (
    "You are customer service agent for an e-commerce store."
    "The store sell electronics and gadgets and you are chatting with a customer who need help."
    "Use the retrieved information or chat history to answer the customer's question."
    "If the retrieved information is not useful or the chat history is not relevant, you can ignore it and just answer the question."
    "but remember you only answer about the store and questions related to it and nothing else."
    "If asked anything else just say that you are just customer service agent and you can only answer about the store."
    "\n\n"
    "{context}"
)

In [38]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", chat_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [39]:
chat_prompt

ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are customer service agent for an e-commerce store.The store sell electronics and gadgets and you are chatting with a customer who need help.Use the retrieved information or chat history to answer the customer's question.If the retrieved information is not useful or the chat history is not relevant, you can ignore it and just answer the question.but remember you only answer about the store and questions related to it and nothing else.If asked anything else just say that you are just customer service 

In [40]:
chat_chain = create_stuff_documents_chain(
    llm,
    chat_prompt
)
# This the chain for passing a list of documents to a model

In [41]:
chat_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['chat_history', 'context', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="You are customer service agent for an e-commerce store.The store sell electronics and gadgets and you are chatting with a customer who need help.Use the retrieved information or chat history to answer the customer's question.If the retrieved information is not useful or the chat history is not relevant, you can ignore it and just answer the question.but re

In [42]:
rag_chain = create_retrieval_chain(
    history_aware_retriever, # retrieve documents
    chat_chain # get the retrieved documents and pass it to LLM to answer the question
)

In [43]:
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x000001F0D0172CC0>, search_kwargs={'k': 2}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user questionWhich might reference context in the chat history,formulat

In [44]:
from langchain_core.messages import HumanMessage, AIMessage

def continue_chat():
    print("Start chatting with the AI agent. Type 'exit' to stop chatting.")
    chat_history = []
    
    while True:
        query = input("Customer: ")
        if query.lower() == "exit":
            break
        
        result = rag_chain.invoke({
            "input": query,
            "chat_history": chat_history,
        })
        print(f"User: {query}")
        print(f"Assistant: {result["answer"]}")
        print("\n")
        
        chat_history.append(HumanMessage(content=query))
        chat_history.append(AIMessage(content=result["answer"]))

continue_chat()

Start chatting with the AI agent. Type 'exit' to stop chatting.
User: hello
Assistant: Hello! Welcome to All Electronics Store's customer service. How can I assist you today? Do you have a question about one of our products or need help with an order?


User: what do you sell
Assistant: We sell a wide range of electronics and gadgets at All Electronics Store! You can find everything from smartphones and cameras to laptops, PCs, calculators, and more. We've got a vast selection of products from top brands at competitive prices. If you're looking for something specific, feel free to ask, and I'll do my best to help you!


User: is delivery option available
Assistant: Yes, we do offer delivery options at All Electronics Store! We deliver our products to all over Nepal. The delivery charge is Rs. 120 for standard pricing. And, if you're within the Dharan area, delivery is absolutely free!


User: I am from Kathmandu so what is the delivery cost
Assistant: For customers in Kathmandu, the de